In [1]:
import json

# 读取 Combined_Personas.json，提取每个代理的 id 和 type
agent_type_map = {}
with open('./config/luban/Combined_Personas.json', 'r') as f:
    personas = json.load(f)
    # 假设 personas 要么是包含代理列表的列表，要么是包含 'agents' 键的字典
    if isinstance(personas, list):
        agents_list = personas
    elif isinstance(personas, dict):
        agents_list = personas.get('agents', personas.get('Agents', personas))
        # 若 personas 本身就是一个单个代理的信息字典，则上行直接返回 personas
        if isinstance(agents_list, dict):
            agents_list = [agents_list]
    else:
        agents_list = []
    # 提取 id 和 type
    for agent in agents_list:
        agent_id = str(agent.get('id'))
        # print(agent_id, agent.get('id'), agent.get('ID'), agent.get('Id'))

        agent_type = agent.get('type') or agent.get('Type')
        if agent_id and agent_type:
            agent_type_map[agent_id] = agent_type




In [2]:
import glob

interactions = []        # 用于存储交互边的信息
attitude_data = {}       # 用于存储每个代理随时间的态度变化
seen_events = set()      # 用于避免重复记录交互事件

# 初始化 attitude_data 字典，包含所有代理的ID键
for aid in agent_type_map.keys():
    attitude_data[aid] = {}

# 遍历所有 ./log/experiment_test1_3_17 下的 agent_*.json 文件
for file in glob.glob('./log/experiment_test1_3_19/agent_*.json'):
    # 从文件名获取 agent ID（假设文件名格式为 agent_<ID>.json）
    print(f"Processing file {file}")
    agent_id = file.split('\\')[1].split('.')[0].split('_')[1]
    with open(file, 'r', encoding='utf-8') as f:
        events = json.load(f)
    
    for event in events:
        ts = event.get('ts')
        event_type = event.get('type')
        item = event.get('item', {})
        
        # 如果存在source和attitude，则为一次交互
        src = item.get('source')
        tgt = agent_id
        att = item.get('attitude')

        # 交互记录
        if src is not None and event['type'] in ["listen", "talk_content"]:
            event_key = (ts, str(src), tgt, event['type'])
            if event_key not in seen_events:
                interactions.append({
                    'source': str(src),
                    'target': tgt,
                    'type': event['type'],
                    'attitude': att,
                    'time': ts
                })
                seen_events.add(event_key)

        # 如果是update或reflect，则记录态度变化
        if event['type'] in ['update', 'reflect', 'vote']:
            new_attitude = item.get('new_attitude') or item.get('attitude')
            if new_attitude is not None:
                print(f"Agent {agent_id} at time {ts} has attitude {new_attitude}")
                attitude_data[tgt][ts] = new_attitude

# 查看解析出的部分交互和态度数据（打印前5条交互和每个代理的初末态度）
print(f"总共提取了 {len(interactions)} 条交互记录。示例几条记录:")
for inter in interactions[:5]:
    print(inter)
print("\n各代理的初始态度和最终态度:")
for aid, times in attitude_data.items():
    if times:
        t_min, t_max = min(times), max(times)
        print(f"Agent {aid} - 初始态度(time {t_min}): {times[t_min]}, 最终态度(time {t_max}): {times[t_max]}")


Processing file ./log/experiment_test1_3_19\agent_0.json
Agent 0 at time 0 has attitude 1.1221786632568262
Agent 0 at time 1 has attitude 7.04712461178447
Agent 0 at time 1 has attitude 7.04712461178447
Agent 0 at time 2 has attitude -22.611094625649272
Agent 0 at time 2 has attitude 9.338959110218017
Agent 0 at time 3 has attitude 39.497334693502
Agent 0 at time 3 has attitude 39.497334693502
Agent 0 at time 4 has attitude -21.60524373486618
Agent 0 at time 4 has attitude -30.915151214571402
Agent 0 at time 5 has attitude -123.49849301561893
Agent 0 at time 5 has attitude -123.49849301561893
Processing file ./log/experiment_test1_3_19\agent_1.json
Agent 1 at time 1 has attitude 0.7
Agent 1 at time 3 has attitude 0.8
Agent 1 at time 5 has attitude 0.89
Processing file ./log/experiment_test1_3_19\agent_10.json
Agent 10 at time 0 has attitude 0.8502780932969389
Agent 10 at time 0 has attitude 0.8502780932969389
Agent 10 at time 1 has attitude 0.8502780932969389
Agent 10 at time 1 has att

In [3]:
# {
#   "agentTypeMap": {
#     "0": "lite",
#     "1": "brief",
#     "2": "full",
#     "3": "lite"
#     // ...
#   },
#   "interactions": [
#     {
#       "source": "0",
#       "target": "1",
#       "type": "listen",
#       "attitude": -28.88,
#       "time": 0
#     },
#     // ...
#   ],
#   "attitudeData": {
#     "0": { "0": -28.88, "1": -137.09, "2": -29.17, "3": -115.15 },
#     "1": { "1": 0.0, "3": 0.0 },
#     "2": { "0": 0.92, "1": 0.92 },
#     // ...
#   }
# }


# 保存数据到 JSON 文件
output_data = {
    "agentTypeMap": agent_type_map,
    "interactions": interactions,
    "attitudeData": attitude_data
}

with open('./优化前_agent_data_2.json', 'w') as f:
    json.dump(output_data, f, indent=2)

